In [3]:
import typing
import os

import pandas
import datasets

import cltrier_lib

import twon_agents

In [4]:
ROOT_PATH: str = "../../.."
EXPORT_NAME: str = "TWON-Agent-OSN-Replies-de"

BASE_MODEL: str = "meta-llama/Llama-3.2-3B-Instruct"

DATASET: str = f"{ROOT_PATH}/data/interim/twitter.german.dataset.csv"

In [14]:
for name, group in pandas.read_csv(DATASET, index_col=0).groupby("author_id_reply"):
    display(group)
    break

,id_post,id_reply,author_id_post,author_id_reply,author_first_name_post,author_last_name_post,author_party_post,text_post,text_reply
0,1616701560431542272,1617114281132314624,42698498,15891726,Roderich,Kiesewetter,CDU/CSU,"Aus meinem Wahlkreis heute eine Bürgerfrage: ""...","Wussten das die CDU Verteidigungsminister, die..."
1,1620171503957581825,1620308520087199744,713361366858481664,15891726,Stefan,Brandner,AfD,Zeigt der #staatsfunk etwa gerade einen #coron...,Da es keinen Staatsfunk gibt: Nein.
2,1649820541207863300,1649890068067151878,18189342,15891726,Frank,Schäffler,FDP,FDP will den Weiterbetrieb der Kernenergie in ...,"Steht da auch drinne, dass die Dinger ohne Sub..."
3,1610260317262471171,1610903010426908673,712941191,15891726,Katja,Adler,FDP,Wir müssen eine Debatte zur Migrationspolitik ...,Leute die Begriffe wie „kulturelle Überfremdun...
4,1657692554706534401,1657718828485206016,713361366858481664,15891726,Stefan,Brandner,AfD,"Jedenfalls haben ""Lord of the Lost""🤪 (der Name...",Oh doch. Zum Beispiel könnte eine AfD Band Nam...
5,1657660680869216256,1657667570541969410,713361366858481664,15891726,Stefan,Brandner,AfD,Der entspannteste Landtagswahlsonntag seit lan...,Stimmt. Hat die Versagertruppe ausnahmsweise m...
6,1646225220376702977,1646586140437217280,403735368,15891726,Florian,Hahn,CDU/CSU,#Cannabis legalisieren und Süßigkeitenwerbung ...,Wenn Sie heulen ist es definitiv das Richtige.
7,1649820541207863300,1649889110750908416,18189342,15891726,Frank,Schäffler,FDP,FDP will den Weiterbetrieb der Kernenergie in ...,Was ewig gestrige halt so beschließen. Wird mi...
8,1636252971993317376,1636416112509304851,18189342,15891726,Frank,Schäffler,FDP,"Die Rettung der Crédit Suisse zeigt, dass die ...",Die machen das so wie Sie.
9,1614664316975828993,1615226444472356865,713361366858481664,15891726,Stefan,Brandner,AfD,Wie fühlt man sich wohl so als letzter #Kliman...,Wie fühlt man sich wohl als Deutschlands dümms...


In [8]:
dataset: typing.List[typing.Dict] = [
    cltrier_lib.inference.schemas.Chat(messages=[
        cltrier_lib.inference.schemas.Message(role="system", content=f"You are a social media user. Respond to the following Tweet based on your last interactions:"),
        cltrier_lib.inference.schemas.Message(role="user", content=row["text_post"]),
        cltrier_lib.inference.schemas.Message(role="assistant", content=row["text_reply"])
    ]).model_dump()
    for _, row in pandas.read_csv(DATASET, index_col=0).iterrows()
]
dataset[:1]

[{'messages': [{'role': 'system',
    'content': 'You are a social media user. Respond to the following Tweet based on your last interactions:'},
   {'role': 'user',
    'content': 'Aus meinem Wahlkreis heute eine Bürgerfrage: "Bloß mal als Frage, warum weiß die Regierung, daß genau 135.000 halbautomatische Gewehre mit Kriegswaffenoptik im Privatbesitz sind, jedoch muß die beim eigenen Bestand von Leoparden erst mal Inventur machen. Das ist doch beschämend."'},
   {'role': 'assistant',
    'content': 'Wussten das die CDU Verteidigungsminister, die nicht mal erklären konnten, wo denn der Wehretat abgeblieben ist?'}]}]

In [3]:
trainer = twon_agents.Trainer(
    name_base_model=BASE_MODEL,
    name_aligned_model=EXPORT_NAME,
    output_dir=f"{ROOT_PATH}/models/{EXPORT_NAME}"
)(dataset)

/home/ubuntu/apis/TWON-Agents/.venv/lib/python3.10/site-packages/transformers/training_args.py:2046: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case simon-muenker/Llama-3.2-3B-Instruct-OSN-replies).
  warnings.warn(
